In [6]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from bs4 import BeautifulSoup

## I have used beautifulsoup library to scrap through the wiki site below

In [9]:
import requests
headers = requests.utils.default_headers()
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XlyyAQpAMEoAALxxwVIAAAAL","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":942851379,"wgRevisionId":942851379,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communi

In [10]:
data = []
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
data

[[],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', 'Downtown Toronto', "Queen's Park"],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 'Etobicoke', 'Martin Grove'],
 ['M9B

In [11]:
data.remove(data[0])

In [12]:
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])

## I have loaded data above and now will do cleaning to create a final DF

In [13]:
df_Toronto = df[df.Borough!='Not assigned']

In [14]:
df_Toronto.reset_index(drop=True)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


In [15]:
df_Toronto.groupby('PostalCode').agg({'Borough':'first', 'Neighborhood': ', '.join}).reset_index()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


## Below is shape of final dataframe

In [16]:
df_Toronto.shape

(210, 3)

## Get the latitude and the longitude coordinates of each neighborhood

In [ ]:
!pip install geocoder
import geocoder # import geocoder

In [ ]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('M9N'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(latitude)
print(longitude)

## I tried geocoder above to test but it is taking forever to get coordinates for a single postal code. So continuing with the coordinate csv

In [17]:
df_coordinates = pd.read_csv('http://cocl.us/Geospatial_data')
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
latseries = pd.Series(['Latitude'])
longseries = pd.Series(['Longitude'])
for index, row in df_Toronto.iterrows():
    PostalCode = row['PostalCode']  
    df_Toronto.loc[index,"Latitude"] = df_coordinates.loc[df_coordinates['Postal Code'] == PostalCode,'Latitude'].item()
    df_Toronto.loc[index,"Longitude"] = df_coordinates.loc[df_coordinates['Postal Code'] == PostalCode,'Longitude'].item()

## Added latitude and longitude details from geolocation csv into DF

In [19]:
df_Toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M6A,North York,Lawrence Heights,43.718518,-79.464763
6,M6A,North York,Lawrence Manor,43.718518,-79.464763
...,...,...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West,43.628841,-79.520999
282,M8Z,Etobicoke,Mimico NW,43.628841,-79.520999
283,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999
284,M8Z,Etobicoke,Royal York South West,43.628841,-79.520999


In [20]:
df_ForToronto=df_Toronto[df_Toronto['Borough'].str.contains("Toronto")]

In [35]:
print(df_ForToronto['Borough'].unique())

['Downtown Toronto' 'East Toronto' 'West Toronto' 'Central Toronto']


## Above we see 4 distinct Boroughs in Toronto

In [22]:
df_ForToronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
7,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
16,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
17,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
33,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
...,...,...,...,...,...
249,M4X,Downtown Toronto,St. James Town,43.667967,-79.367675
250,M5X,Downtown Toronto,First Canadian Place,43.648429,-79.382280
251,M5X,Downtown Toronto,Underground city,43.648429,-79.382280
261,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160


## Segregated just the Boroughs with Toronto in it above. Now will explore and cluster below

In [23]:
import folium

In [25]:
!pip install geopy
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

In [26]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\ppratish.HM-ROA\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [32]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_ForToronto['Latitude'], df_ForToronto['Longitude'], df_ForToronto['Borough'], df_ForToronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [74]:
CLIENT_ID = 'BOTFOOJQLGKTDTFJVR2GRUX0J1SCMHH0G5DUHP1SNVLC5BCM' # your Foursquare ID
CLIENT_SECRET = '5SDO5WRZSXLD5GA3Y13UCBKWYJLUA5UZOLB4EGSFPXUYJVIA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [34]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

Libraries imported.


## Lets start by exploring downtown Toronto

In [37]:
dToronto_data = df_ForToronto[df_ForToronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [38]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of downtown Toronto using latitude and longitude values
map_dtoronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(dToronto_data['Latitude'], dToronto_data['Longitude'], dToronto_data['Borough'], dToronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dtoronto)  
    
map_dtoronto

C:\Users\ppratish.HM-ROA\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


## Lets create a function to explore each neighborhood first

In [62]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT = 100 # limit of number of venues returned by Foursquare API
        radius = 500 # define radius    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'id',           
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Now lets get data for all venues in downtown Toronto

In [ ]:
Toronto_venues = getNearbyVenues(names=df_ForToronto['Neighborhood'],
                                   latitudes=df_ForToronto['Latitude'],
                                   longitudes=df_ForToronto['Longitude']
                                  )

In [64]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))
Toronto_venues['Venue Category'].unique()

There are 236 uniques categories.


array(['Bakery', 'Coffee Shop', 'Gym / Fitness Center', 'Spa',
       'Breakfast Spot', 'Restaurant', 'Park', 'Historic Site', 'Pub',
       'Farmers Market', 'Chocolate Shop', 'Dessert Shop', 'Theater',
       'Performing Arts Venue', 'French Restaurant', 'Café',
       'Mexican Restaurant', 'Event Space', 'Greek Restaurant',
       'Yoga Studio', 'Ice Cream Shop', 'Shoe Store', 'Asian Restaurant',
       'Art Gallery', 'Cosmetics Shop', 'Electronics Store', 'Bank',
       'Beer Store', 'Hotel', 'Health Food Store', 'Wine Shop',
       'Antique Shop', 'Portuguese Restaurant', 'Italian Restaurant',
       'Gym', 'Sushi Restaurant', 'Creperie', 'Beer Bar', 'Burrito Place',
       'Arts & Crafts Store', 'Burger Joint', 'Diner',
       'Fried Chicken Joint', 'Japanese Restaurant', 'Discount Store',
       'Wings Joint', 'Nightclub', 'Fast Food Restaurant',
       'Seafood Restaurant', 'Chinese Restaurant', 'Juice Bar',
       'Sandwich Place', 'College Auditorium', 'Bar', 'Music Venue',
 

## We see 236 categories of venues here but i will remove the one's which dont interest me

In [65]:
removethese = ['Spa','Breakfast Spot', 'Restaurant', 'Chocolate Shop', 'Dessert Shop', 'Theater',
       'Performing Arts Venue', 'French Restaurant', 'Café',
       'Mexican Restaurant', 'Event Space', 'Greek Restaurant',
       'Ice Cream Shop', 'Shoe Store', 'Asian Restaurant',
       'Art Gallery', 'Cosmetics Shop', 'Bank',
       'Beer Store', 'Hotel', 'Health Food Store', 
       'Antique Shop', 'Portuguese Restaurant', 
       'Gym', 'Sushi Restaurant', 'Creperie', 'Beer Bar', 'Burrito Place',
       'Arts & Crafts Store', 'Burger Joint', 'Diner',
       'Fried Chicken Joint', 'Japanese Restaurant', 'Discount Store',
       'Wings Joint', 'Nightclub', 'Fast Food Restaurant',
       'Seafood Restaurant', 'Chinese Restaurant', 
       'Sandwich Place', 'College Auditorium', 'Bar', 'Music Venue',
       'Comic Shop', 'Pizza Place', 'Plaza', 'Tea Room',
       'Ramen Restaurant', 'Thai Restaurant', 
       'Sporting Goods Shop', 'Steakhouse',
       'American Restaurant', 'Gastropub', 'Tanning Salon',
       'College Rec Center', 'Miscellaneous Shop',
       'Modern European Restaurant', 'Middle Eastern Restaurant', 'Lake',
       'Video Game Store', 'Furniture / Home Store', 'Lounge',
       'Bubble Tea Shop', 'Department Store', 'Distribution Center',
       'Toy / Game Store', 'Ethiopian Restaurant', 'Other Great Outdoors',
       'Lingerie Store', 'Speakeasy', 'Wine Bar', 'Office',
       'Poutine Place', 'BBQ Joint', 'Food Truck', 'Church', 'Poke Place',
       'Hostel', 'New American Restaurant', 'Latin American Restaurant',
       'Cocktail Bar', 'Camera Store',
       'Fountain', 'Tailor Shop', 'Fish Market',
       'Cheese Shop', 'Bagel Shop', 'German Restaurant',
       'Food & Drink Shop', 'Butcher', 'Comfort Food Restaurant',
       'Salon / Barbershop', 'Trail', 'Neighborhood', 'Liquor Store',
       'Museum', 'Bistro', 'Basketball Stadium',
       'Jazz Club', 'Beach', 'Eastern European Restaurant',
       'Gourmet Shop', 'Irish Pub', 'Donut Shop', 'Salad Place',
       'Korean Restaurant', 'Candy Store', 
       'Opera House', 'Food Court', 'Smoke Shop', 'Noodle House',
       'General Travel', 'Monument / Landmark', 'Colombian Restaurant',
       'Deli / Bodega', 'Brazilian Restaurant', 'Record Shop',
       'Gluten-free Restaurant', 'Mediterranean Restaurant',
       "Women's Store", 'Building', 'Brewery', 
       'Skating Rink', 'Sports Bar', 'IT Services', 'Dance Studio',
       'Roof Deck', 'Aquarium', 'Scenic Lookout',
       'History Museum', 'Baseball Stadium', 'Hotel Bar',
       'Cuban Restaurant', 'Vietnamese Restaurant', 'Mac & Cheese Joint',
       "Men's Store", 'Cupcake Shop', 'Gift Shop',
       'Caribbean Restaurant',
       'Frozen Yogurt Shop', 'Pet Store', 'Climbing Gym', 'Stadium',
       'Intersection', 'Convenience Store', 'Fish & Chips Shop',
       'Gay Bar', 'Stationery Store', 'Coworking Space',
       'Thrift / Vintage Store', 'Swim School', 
       'Dog Run', 'Jewelry Store', 'Flea Market',
       'Cajun / Creole Restaurant', 'Flower Shop',
       'Costume Shop', 'Indoor Play Area', 'College Gym',
       'College Arts Building', 'Moving Target', 'Food',
       'Supplement Shop',
       'South American Restaurant', 'Tennis Court', 
       'Organic Grocery', 'Belgian Restaurant', 'Dumpling Restaurant',
       'Gaming Cafe', 'Hotpot Restaurant', 'Doner Restaurant',
       'Filipino Restaurant', 'Dim Sum Restaurant', 'Massage Studio',
       'Bed & Breakfast', 'Light Rail Station', 'Airport',
       'Airport Lounge', 'Harbor / Marina', 'Airport Food Court',
       'Airport Terminal', 'Airport Gate', 'Boutique', 'Airport Service',
       'Rental Car Location', 'Sculpture Garden', 'Boat or Ferry',
       'Molecular Gastronomy Restaurant', 'Optical Shop', 'Soup Place',
       'Taiwanese Restaurant', 'Market',
       'Snack Place', 'Theme Restaurant', 'Sake Bar', 'Afghan Restaurant',
       'Health & Beauty Service', 'Strip Club', 'Skate Park',
       'Garden Center', 'Auto Workshop']

Toronto_myvenues = Toronto_venues.copy()
Toronto_myvenues = Toronto_myvenues[~Toronto_myvenues['Venue Category'].isin(removethese)]
Toronto_myvenues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,id,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.654260,-79.360636,54ea41ad498e9a11e9e13308,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.654260,-79.360636,53b8466a498e83df908c3f21,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.654260,-79.360636,574c229e498ebb5c6b257902,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
6,Harbourfront,43.654260,-79.360636,51ccc048498ec7792efc955e,Corktown Common,43.655618,-79.356211,Park
8,Harbourfront,43.654260,-79.360636,4ad4c05ef964a520bff620e3,The Distillery Historic District,43.650244,-79.359323,Historic Site
...,...,...,...,...,...,...,...,...
3228,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,4deb8ba688774880e3387c0c,Leslieville Farmers Market,43.664901,-79.319784,Farmers Market
3237,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,4be3345b660ec928226dcb3b,The Ashbridge Estate,43.664691,-79.321805,Garden
3239,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,4d0cef321f6bf04d75707a31,Jonathan Ashbridge Park,43.664702,-79.319898,Park
3241,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,5131f59ae4b0ba071c79c64d,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio


In [66]:
Toronto_myvenues['Venue Category'].unique()

array(['Bakery', 'Coffee Shop', 'Gym / Fitness Center', 'Park',
       'Historic Site', 'Pub', 'Farmers Market', 'Yoga Studio',
       'Electronics Store', 'Wine Shop', 'Italian Restaurant',
       'Juice Bar', 'Clothing Store', 'Movie Theater', 'Shopping Mall',
       'Bookstore', 'Vegetarian / Vegan Restaurant', 'Grocery Store',
       'Concert Hall', 'Pharmacy', 'Falafel Restaurant',
       'Indian Restaurant', 'Baby Store', 'Gas Station', 'Supermarket',
       'Pool', 'Train Station', 'Fruit & Vegetable Store', 'Bus Line',
       'Garden', 'Metro Station', 'Indie Movie Theater', 'Playground',
       'Hospital', 'General Entertainment'], dtype=object)

## Now I have my list of venues based on which I can cluster neighborhoods

In [67]:
myVenues_id = Toronto_myvenues['id'].tolist()

In [85]:
myurl = []
myresp = []
myjson = []

for i in myVenues_id:
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    myurl.append(venue_url)
for link in myurl:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    myresp.append(likes)
print(myresp)

[47, 49, 18, 43, 546, 32, 101, 240, 57, 19, 15, 56, 26, 16, 17, 16, 8, 416, 6, 31, 8, 0, 1, 170, 56, 3, 10, 15, 14, 9, 18, 14, 1, 0, 2, 0, 0, 35, 114, 7, 5, 782, 23, 2993, 84, 8, 102, 37, 35, 244, 106, 172, 6, 26, 12, 16, 30, 127, 9, 496, 54, 23, 75, 51, 42, 8, 1, 114, 7, 5, 782, 23, 2993, 84, 8, 102, 37, 35, 244, 106, 172, 6, 26, 12, 16, 30, 127, 9, 496, 54, 23, 75, 51, 42, 8, 1, 275, 229, 34, 25, 55, 69, 134, 66, 53, 1272, 11, 48, 11, 71, 234, 39, 54, 244, 6, 496, 30, 101, 51, 6, 23, 45, 16, 11, 267, 53, 1272, 134, 11, 8, 54, 10, 7, 171, 78, 101, 6, 155, 163, 41, 3, 35, 9, 56, 10, 69, 172, 15, 58, 14, 22, 23, 12, 13, 37, 10, 54, 11, 42, 1, 0, 1, 1, 20, 4, 5, 89, 6, 35, 59, 24, 0, 4, 0, 1, 1, 222, 29, 32, 17, 21, 15, 14, 16, 40, 11, 102, 81, 320, 244, 28, 12, 496, 45, 55, 30, 51, 222, 29, 32, 17, 21, 15, 14, 16, 40, 11, 102, 81, 320, 244, 28, 12, 496, 45, 55, 30, 51, 222, 29, 32, 17, 21, 15, 14, 16, 40, 11, 102, 81, 320, 244, 28, 12, 496, 45, 55, 30, 51, 25, 29, 4, 8, 7, 4, 0, 3, 0, 5

In [86]:
print(len(myresp))
print(len(myVenues_id))

848
848


In [87]:
toronto_venues = Toronto_myvenues.copy()

In [89]:
toronto_venues['totallikes'] = myresp
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,id,Venue,Venue Latitude,Venue Longitude,Venue Category,totallikes
0,Harbourfront,43.65426,-79.360636,54ea41ad498e9a11e9e13308,Roselle Desserts,43.653447,-79.362017,Bakery,47
1,Harbourfront,43.65426,-79.360636,53b8466a498e83df908c3f21,Tandem Coffee,43.653559,-79.361809,Coffee Shop,49
2,Harbourfront,43.65426,-79.360636,574c229e498ebb5c6b257902,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center,18
6,Harbourfront,43.65426,-79.360636,51ccc048498ec7792efc955e,Corktown Common,43.655618,-79.356211,Park,43
8,Harbourfront,43.65426,-79.360636,4ad4c05ef964a520bff620e3,The Distillery Historic District,43.650244,-79.359323,Historic Site,546


In [90]:
print(toronto_venues['totallikes'].max())
print(toronto_venues['totallikes'].min())
print(toronto_venues['totallikes'].median())
print(toronto_venues['totallikes'].mean())


2993
0
23.0
62.472877358490564


In [92]:
print(np.percentile(toronto_venues['totallikes'], 25))
print(np.percentile(toronto_venues['totallikes'], 50))
print(np.percentile(toronto_venues['totallikes'], 75))

9.0
23.0
54.0


## Now we will create separate categories based on % no. of likes to categorize our neighborhoods 

In [95]:
really_bad = toronto_venues['totallikes']<=24
will_manage = toronto_venues[(toronto_venues['totallikes']>24) & (toronto_venues['totallikes']<=45)]
second_best = toronto_venues[(toronto_venues['totallikes']>45) & (toronto_venues['totallikes']<=76)]
the_best = toronto_venues['totallikes']>76

In [97]:
# one hot encoding
to_onehot = pd.get_dummies(toronto_venues[['Venue Category', 'totallikes']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Name'] = toronto_venues['Venue'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

to_onehot.head()

,Name,totallikes,Baby Store,Bakery,Bookstore,Bus Line,Clothing Store,Coffee Shop,Concert Hall,Electronics Store,...,Pharmacy,Playground,Pool,Pub,Shopping Mall,Supermarket,Train Station,Vegetarian / Vegan Restaurant,Wine Shop,Yoga Studio
0,Roselle Desserts,47,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Tandem Coffee,49,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cooper Koo Family YMCA,18,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Corktown Common,43,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,The Distillery Historic District,546,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## We now start k-mean clustering based on categories defined above

In [99]:
cluster_df = to_onehot.drop('Name', axis=1)

k_clusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(cluster_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 3, 0, 0, 3, 0, 0])

In [101]:
toronto_venues['label'] = kmeans.labels_
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,id,Venue,Venue Latitude,Venue Longitude,Venue Category,totallikes,label
0,Harbourfront,43.65426,-79.360636,54ea41ad498e9a11e9e13308,Roselle Desserts,43.653447,-79.362017,Bakery,47,0
1,Harbourfront,43.65426,-79.360636,53b8466a498e83df908c3f21,Tandem Coffee,43.653559,-79.361809,Coffee Shop,49,0
2,Harbourfront,43.65426,-79.360636,574c229e498ebb5c6b257902,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center,18,0
6,Harbourfront,43.65426,-79.360636,51ccc048498ec7792efc955e,Corktown Common,43.655618,-79.356211,Park,43,0
8,Harbourfront,43.65426,-79.360636,4ad4c05ef964a520bff620e3,The Distillery Historic District,43.650244,-79.359323,Historic Site,546,3


In [103]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [104]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i+x+(i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_venues['Venue Latitude'], toronto_venues['Venue Longitude'], toronto_venues['Venue'], toronto_venues['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1 below

In [105]:
toronto_venues.loc[toronto_venues['label']==0]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,id,Venue,Venue Latitude,Venue Longitude,Venue Category,totallikes,label
0,Harbourfront,43.654260,-79.360636,54ea41ad498e9a11e9e13308,Roselle Desserts,43.653447,-79.362017,Bakery,47,0
1,Harbourfront,43.654260,-79.360636,53b8466a498e83df908c3f21,Tandem Coffee,43.653559,-79.361809,Coffee Shop,49,0
2,Harbourfront,43.654260,-79.360636,574c229e498ebb5c6b257902,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center,18,0
6,Harbourfront,43.654260,-79.360636,51ccc048498ec7792efc955e,Corktown Common,43.655618,-79.356211,Park,43,0
9,Harbourfront,43.654260,-79.360636,566e1294498e3f6629006bc3,Dominion Pub and Kitchen,43.656919,-79.358967,Pub,32,0
...,...,...,...,...,...,...,...,...,...,...
3228,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,4deb8ba688774880e3387c0c,Leslieville Farmers Market,43.664901,-79.319784,Farmers Market,33,0
3237,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,4be3345b660ec928226dcb3b,The Ashbridge Estate,43.664691,-79.321805,Garden,4,0
3239,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,4d0cef321f6bf04d75707a31,Jonathan Ashbridge Park,43.664702,-79.319898,Park,1,0
3241,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,5131f59ae4b0ba071c79c64d,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio,6,0


## Cluster 2 below

In [106]:
toronto_venues.loc[toronto_venues['label']==1]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,id,Venue,Venue Latitude,Venue Longitude,Venue Category,totallikes,label
112,Ryerson,43.657162,-79.378937,4ad77a12f964a520260b21e3,CF Toronto Eaton Centre,43.65454,-79.380677,Shopping Mall,2993,1
212,Garden District,43.657162,-79.378937,4ad77a12f964a520260b21e3,CF Toronto Eaton Centre,43.65454,-79.380677,Shopping Mall,2993,1


## Cluster 3 below

In [107]:
toronto_venues.loc[toronto_venues['label']==2]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,id,Venue,Venue Latitude,Venue Longitude,Venue Category,totallikes,label
106,Ryerson,43.657162,-79.378937,4adbbae6f964a520402a21e3,Cineplex Cinemas,43.656126,-79.380390,Movie Theater,782,2
206,Garden District,43.657162,-79.378937,4adbbae6f964a520402a21e3,Cineplex Cinemas,43.656126,-79.380390,Movie Theater,782,2
334,St. James Town,43.651494,-79.375418,4ad4c062f964a520fbf720e3,St. Lawrence Market (South Building),43.648743,-79.371597,Farmers Market,1272,2
402,Berczy Park,43.644771,-79.373306,4ad4c062f964a520fbf720e3,St. Lawrence Market (South Building),43.648743,-79.371597,Farmers Market,1272,2
2776,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,4ad4c062f964a520fbf720e3,St. Lawrence Market (South Building),43.648743,-79.371597,Farmers Market,1272,2


## Cluster 4 below

In [108]:
toronto_venues.loc[toronto_venues['label']==3]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,id,Venue,Venue Latitude,Venue Longitude,Venue Category,totallikes,label
8,Harbourfront,43.654260,-79.360636,4ad4c05ef964a520bff620e3,The Distillery Historic District,43.650244,-79.359323,Historic Site,546,3
12,Harbourfront,43.654260,-79.360636,51853a73498e4d97a8b20831,Rooster Coffee,43.651900,-79.365609,Coffee Shop,240,3
32,Harbourfront,43.654260,-79.360636,4af9fc95f964a520ca1522e3,Mill St. Brew Pub,43.650353,-79.358489,Pub,416,3
50,Queen's Park,43.662301,-79.389494,4b9d206bf964a520e69136e3,Queen's Park,43.663946,-79.392180,Park,170,3
138,Ryerson,43.657162,-79.378937,4b2a6eb8f964a52012a924e3,Indigo,43.653515,-79.380696,Bookstore,244,3
...,...,...,...,...,...,...,...,...,...,...
3033,First Canadian Place,43.648429,-79.382280,4adc55c6f964a520bd2b21e3,Starbucks,43.649328,-79.378339,Coffee Shop,234,3
3077,Underground city,43.648429,-79.382280,4ad4c062f964a520e5f720e3,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall,222,3
3082,Underground city,43.648429,-79.382280,4ad4c061f964a520b0f720e3,Roy Thomson Hall,43.646589,-79.385979,Concert Hall,320,3
3114,Underground city,43.648429,-79.382280,55561f8a498e2e5c866ac4c9,Union Pearson Express,43.644362,-79.383199,Train Station,148,3


## Based on above clustering, it is clear that as per my requirement:

### 1. Best Neighborhoods are: Ryerson, Garden District
### 2. Second Best Neighborhoods are: St. James Town, Berczy Park, Stn A PO Boxes 25 The Esplanade